In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import os
import time

# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('mode.chained_assignment', 'raise')SimpleImputerSimpleImputerSimpleImputer

In [2]:
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
        
train = pd.read_csv('../suspicious-transaction-detection/train.csv')
# test = pd.read_csv('suspicious-transaction-detection/test.csv')

N_train, dim = train.shape
# N_test, _ = test.shape

# X_train = train.drop(['Target'], axis=1)
# y_train = train['Target'].copy()

In [3]:
med_features = np.concatenate((['Amount'],
                               ['T_{}'.format(t) for t in range(15)], 
                               ['C_{}'.format(c) for c in range(9, 23)],
                               ['C_26', 'C_27', 'C_28'], 
                               ['V_{}'.format(v) for v in range(339)], 
                               ['O_0', 'O_1', 'O_5', 'O_6', 'O_9', 'O_18', 'O_31', 'O_36'],
                               ['A_0', 'M_1']
                              ))
mod_features = np.concatenate((['C_0', 'C_1', 'C_2', 'C_3', 'C_4', 'C_6', 'C_7', 'C_8', 'C_23'],
                               ['O_2', 'O_3', 'O_10', 'O_11', 'O_12', 'O_15', 'O_20', 'O_21', 'O_22', 'O_24', 'O_26', 'O_27', 'O_28', 'O_29', 'O_30', 'O_32', 'O_33', 'O_34', 'O_35', 'O_37', 'O_38'],
                               ['A_1']
                              ))
oh_features = np.concatenate((['Goods', 'C_5', 'C_24', 'C_25'],
                              ['O_4', 'O_7', 'O_8', 'browser', 'O_16', 'O_17', 'os', 'O_23', 'O_25',  'O_39'],
                              ['E_same', 'M_0']
                             ))

print(len(med_features) + len(mod_features) + len(oh_features))

429


In [4]:
# features = pd.concat([X_train, test]).reset_index(drop=True)
features = train

features = features.drop(['TransactionID', 'Timestamp', 'O_14'], axis=1)

features['browser'] = features['O_13'].str.split(' ').str[0]
features['os'] = features['O_19'].str.split(' ').str[0]
features['E_0'] = features['E_0'].fillna('empty')
features['E_same'] = np.where(features['E_0'] == features['E_1'], 'T', 'F')

features = features.drop(['O_13', 'O_19', 'E_0', 'E_1'], axis=1)

for feature in med_features:
    med = train[feature].median()
    features[feature] = features[feature].fillna(med)
for feature in mod_features:
    mod = train[feature].mode()[0]
    #print('{}: {}'.format(feature, mod))
    features[feature] = features[feature].fillna(mod)

features_prepared = pd.get_dummies(features, columns=oh_features)

features_prepared.replace('T', 1, inplace=True)
features_prepared.replace('F', 0, inplace=True)




# xTr = 

# X_train_prepared = features_prepared.iloc[:N_train, :].copy()
# X_test_prepared = features_prepared.iloc[N_train:, :].copy()

# print(X_train_prepared.shape)
# print(X_test_prepared.shape)

In [5]:
features_prepared

,Target,Amount,T_0,T_1,T_2,T_3,T_4,T_5,T_6,T_7,...,O_23_NotFound,O_25_Found,O_25_New,O_39_Found,O_39_New,O_39_Unknown,E_same_F,E_same_T,M_0_D,M_0_M
0,0,58.14,30.0,30.0,30.0,0.0,38.041664,0.0,0.0,30.0,...,0,0,0,0,0,0,1,0,0,0
1,0,47.07,4.0,4.0,213.0,0.0,38.041664,0.0,0.0,367.0,...,0,0,0,0,0,0,1,0,0,0
2,0,22.31,97.0,0.0,0.0,0.0,0.000000,0.0,0.0,43.0,...,1,1,0,1,0,0,0,1,1,0
3,0,149.94,97.0,0.0,52.0,0.0,21.958332,0.0,0.0,43.0,...,1,1,0,1,0,0,0,1,0,1
4,0,117.01,129.0,129.0,130.0,0.0,38.041664,0.0,0.0,43.0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472427,0,39.11,277.0,277.0,277.0,0.0,38.041664,0.0,0.0,43.0,...,0,0,0,0,0,0,1,0,0,0
472428,0,30.77,310.0,310.0,484.0,0.0,38.041664,0.0,0.0,43.0,...,0,0,0,0,0,0,1,0,0,0
472429,0,107.84,188.0,188.0,365.0,0.0,38.041664,0.0,0.0,358.0,...,0,0,0,0,0,0,1,0,0,0
472430,0,30.02,108.0,108.0,166.0,0.0,38.041664,0.0,0.0,166.0,...,0,0,0,0,0,0,1,0,0,0


In [6]:
from sklearn.decomposition import PCA
from sklearn import preprocessing

def split_xy(df):
    y = df['Target']
    x = df.drop('Target',axis=1)
    return x,y

x,y = split_xy(features_prepared)
c = x.columns
standard = preprocessing.StandardScaler()
x = pd.DataFrame(standard.fit_transform(x),columns=c)
x

,Amount,T_0,T_1,T_2,T_3,T_4,T_5,T_6,T_7,T_8,...,O_23_NotFound,O_25_Found,O_25_New,O_39_Found,O_39_New,O_39_Unknown,E_same_F,E_same_T,M_0_D,M_0_M
0,-0.318816,-0.787649,-0.407259,-0.611554,-0.083369,-0.151995,-0.133459,-0.156047,-0.467473,-0.242302,...,-0.513623,-0.385176,-0.351328,-0.360107,-0.341489,-0.142311,0.458733,-0.458733,-0.410647,-0.323016
1,-0.364853,-0.982356,-0.572246,0.346450,-0.083369,-0.151995,-0.133459,-0.156047,1.863558,-0.321214,...,-0.513623,-0.385176,-0.351328,-0.360107,-0.341489,-0.142311,0.458733,-0.458733,-0.410647,-0.323016
2,-0.467824,-0.285902,-0.597628,-0.768604,-0.083369,-0.575649,-0.133459,-0.156047,-0.377552,-0.400126,...,1.946952,2.596218,-0.351328,2.776954,-0.341489,-0.142311,-2.179917,2.179917,2.435180,-0.323016
3,0.062958,-0.285902,-0.597628,-0.496384,-0.083369,-0.331109,-0.133459,-0.156047,-0.377552,-0.242302,...,1.946952,2.596218,-0.351328,2.776954,-0.341489,-0.142311,-2.179917,2.179917,-0.410647,3.095820
4,-0.073990,-0.046262,0.220960,-0.088055,-0.083369,-0.151995,-0.133459,-0.156047,-0.377552,1.493758,...,-0.513623,-0.385176,-0.351328,-0.360107,-0.341489,-0.142311,0.458733,-0.458733,-0.410647,-0.323016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472427,-0.397957,1.062074,1.160115,0.681490,-0.083369,-0.151995,-0.133459,-0.156047,-0.377552,-0.368561,...,-0.513623,-0.385176,-0.351328,-0.360107,-0.341489,-0.142311,0.458733,-0.458733,-0.410647,-0.323016
472428,-0.432641,1.309203,1.369521,1.765134,-0.083369,-0.151995,-0.133459,-0.156047,-0.377552,-0.242302,...,-0.513623,-0.385176,-0.351328,-0.360107,-0.341489,-0.142311,0.458733,-0.458733,-0.410647,-0.323016
472429,-0.112125,0.395575,0.595352,1.142170,-0.083369,-0.151995,-0.133459,-0.156047,1.801304,-0.242302,...,-0.513623,-0.385176,-0.351328,-0.360107,-0.341489,-0.142311,0.458733,-0.458733,-0.410647,-0.323016
472430,-0.435760,-0.203526,0.087701,0.100405,-0.083369,-0.151995,-0.133459,-0.156047,0.473239,-0.273867,...,-0.513623,-0.385176,-0.351328,-0.360107,-0.341489,-0.142311,0.458733,-0.458733,-0.410647,-0.323016


In [7]:


# max_scaler = preprocessing.MaxAbsScaler()
# standard = preprocessing.StandardScaler()
# features_prepared = standard.fit_transform(features_prepared)
# X_train_max = max_scaler.fit_transform(X_train_standard)

pca = PCA(n_components=3)
pc = pd.DataFrame(pca.fit_transform(x))
pc['Target'] = y
print(pc)
validation_data = pc.sample(frac=.1)
pre_train_data = pc.drop(validation_data.index).reset_index(drop=True)
xTr, yTr = split_xy(pre_train_data)
xTe, yTe = split_xy(validation_data)




ValueError: could not convert string to float: 'A'

In [ ]:
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
# from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score

In [ ]:
grid = {}
grid['knn'] = {'p':[1,2], 'n_neighbors':[1,2,3,4,5,6,7,8,9]}

grid['rf'] = {'n_estimators': [50, 100], 'max_depth':[None,2,4,6]}

# grid['svmp'] = {'kernel':['poly'],'degree':[2,3], 'gamma':['scale'], 'coef0':[.01,.1,1,5,10,20], 'C':[.01,.1,.5,1], 'tol':[1e-1,1e-2,1e-3,1e-4]}
grid['svmg'] = {'kernel':['rbf'],'gamma':['auto'],'C':[.01,.1,.5,1], 'tol':[1e-1,1e-2,1e-3,1e-4]}

# grid['nnrelu'] = {'hidden_layer_sizes':[(10,30,10),(10,10,10)],'activation':["relu"],   'learning_rate':['constant','adaptive'],'max_iter':[100,500,800],'alpha':[.1,.01,.001,.0001,.00001],'tol':[1e-2,1e-3,1e-4,1e-5],'beta_1':[.01,.1,.25,.5,.7,.9],'beta_2':[.015,.1,.25,.4,.65,.9],'epsilon':[1e-4,1e-5,1e-6,1e-7]}
# grid['nnsig'] = {'hidden_layer_sizes':[(10,30,10),(10,10,10)],'activation':["logistic"],'learning_rate':['constant','adaptive'],'max_iter':[100,500,800],'learning_rate_init':[.01,.001,.0001],'alpha':[.1,.01,.001,.0001,.00001],'tol':[1e-2,1e-3,1e-4,1e-5],'beta_1':[.01,.1,.25,.5,.7,.9],'beta_2':[.015,.1,.25,.4,.65,.9],'epsilon':[1e-4,1e-5,1e-6,1e-7]}

In [ ]:
cv = KFold(n_splits=3, random_state=1, shuffle=True).split(pre_train_data)

clf_knn = GridSearchCV(estimator=KNeighborsClassifier(), param_grid=grid['knn'], n_jobs=-1, cv=cv)
clf_knn.fit(xTr, yTr)
preds_knn = clf_knn.predict_proba(xTe)
roc_knn = roc_auc_score(yTe,preds_knn)
print("KNN AUC : %s - %s" % (roc_knn,clf_knn.best_params_))

clf_rf = GridSearchCV(estimator=RandomForestClassifier(), param_grid=grid['rf'], n_jobs=-1, cv=cv)
clf_rf.fit(xTr, yTr)
preds_rf = clf_rf.predict_proba(xTe)
roc_rf = roc_auc_score(yTe,preds_rf)
print("RF AUC : %s - %s" % (roc_rf,clf_rf.best_params_))

# clf_svmp = GridSearchCV(estimator=svm.SVC(), param_grid=grid['svmp'], n_jobs=-1, cv=cv)
# clf_svmp.fit(xTr, yTr)
# preds_svmp = clf_svmp.predict_proba(xTe)
# roc_svmp = roc_auc_score(yTe,preds_svmp)
# print("SVM[P] AUC : %s - %s" % (roc_svmp,clf_svmp.best_params_))

clf_svmg = GridSearchCV(estimator=svm.SVC(), param_grid=grid['svmg'], n_jobs=-1, cv=cv)
clf_svmg.fit(xTr, yTr)
preds_svmg = clf_svmg.predict_proba(xTe)
roc_svmg = roc_auc_score(yTe,preds_svmg)
print("SVM[G] AUC : %s - %s" % (roc_svmg,clf_svmg.best_params_))